In [1]:
from smolgrad import Tensor
from smolgrad.core import no_grad
import smolgrad.nn as nn

In [2]:
n_embd = 10
block_size = 12
vocab_size = 5

model = nn.ModuleDict(dict(
    wte = nn.Embedding(vocab_size, n_embd),
    # wpe = nn.Embedding(block_size, n_embd),
    # h = nn.ModuleList([nn.Linear(n_embd, n_embd) for _ in range(3)]),
    # ln_f = nn.LayerNorm(n_embd)
))
model.eval()

In [3]:
input_tensor = Tensor([[1, 2, 3, 4, 0], [2, 4, 3, 0, 1]])
print(input_tensor.shape)

with no_grad():
    output = model(input_tensor)

(2, 5)


In [4]:
output.shape

(2, 5, 10)

In [5]:
output

Tensor(array([[[0.360508, 0.111439, -0.0733204, ..., -1.65297, 0.342735, 0.871474],
        [0.977734, 2.28909, 0.82623, ..., 1.62936, -0.947261, 0.556224],
        [-0.77323, 0.329918, 1.14783, ..., -1.64918, -0.443001, 0.692393],
        [1.28988, -0.0404159, 1.21334, ..., 0.497808, -0.494038, -0.0139012],
        [-1.94214, -0.209115, -1.64965, ..., 0.660396, -0.698628, -0.837091]],
       [[0.977734, 2.28909, 0.82623, ..., 1.62936, -0.947261, 0.556224],
        [1.28988, -0.0404159, 1.21334, ..., 0.497808, -0.494038, -0.0139012],
        [-0.77323, 0.329918, 1.14783, ..., -1.64918, -0.443001, 0.692393],
        [-1.94214, -0.209115, -1.64965, ..., 0.660396, -0.698628, -0.837091],
        [0.360508, 0.111439, -0.0733204, ..., -1.65297, 0.342735, 0.871474]]], dtype=float32), is_mlx_tensor=True)

In [6]:
final = output[1:, :5]
final.shape

(1, 5, 10)